<a href="https://colab.research.google.com/github/mahsaghaderan99/MLP/blob/backpropagation/mlp123.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Deep MLP project

#Implement MLP

In [1]:
import tensorflow as tf
from keras.utils.np_utils import to_categorical  
from os import path, getcwd, chdir
import numpy as np

In [110]:
x = np.linspace(0,1,200,dtype=np.float16)
np.random.shuffle(x)


In [137]:
class Layer:
  def __init__(self,node_number,activation):
    
    self.node_num = node_number
    self.delta = np.empty(node_number)
    self.vj = np.empty(node_number)
    self.outj = np.empty(node_number)
    self.b = np.zeros((self.node_num, 1)) 
    self.activation = self._sigmoid if activation == 'sigmoid' else self._relu
    self.d_activation = self._d_sigmoid if activation == 'sigmoid' else self._d_relu

  def define_weights(self,input_shape):
    self.weights = np.random.uniform(low=0, high=1, size=(self.node_num,input_shape))

  def _sigmoid(self,x):
    return np.power(1+np.exp(-0.5*x),-1)
    
  def _d_sigmoid(self,x):
    return self._sigmoid(x)*(1-self._sigmoid(x))

  def _relu(self,x):
    return x*(x>0)

  def _d_relu(self,x):
    return 1.*(x>0)
   

In [141]:
import numpy as np

class Model:
  def __init__(self,learning_rate,epochs):
    self.layers = np.empty(0,dtype=Layer)
    self.lr = learning_rate
    self.epochs = epochs

  def Dense(self,node_number,activation_function):
    self.layers = np.append(self.layers,Layer(node_number,activation_function))

  def Train(self,training,training_y):
    try:
      self.training = training.reshape(training.shape[0],training.shape[1]*training.shape[2])
    except:
      self.training = np.expand_dims(training,1)
    self.desiered_labels = training_y
    self._define_weights()
    while self.epochs>0:
      print(self.epochs)
      self.epochs -=1
      for tr in range(0,self.training.shape[0]):
        yhat = self._forward_propagation(tr)
        self._update_weights(yhat,tr)

  def _define_weights(self):
    self.layers[0].define_weights(self.training[0].shape[0])
    for i in range(1,self.layers.shape[0]):
      self.layers[i].define_weights(self.layers[i-1].node_num)

  def _forward_propagation(self,tr):
    yhat = self._layer_output(self.training[tr] ,0)
    for i in range(1,self.layers.shape[0]):
      yhat = self._layer_output(yhat ,i)
    # print( yhat)
    return yhat

  def _layer_output(self,x,i):
    yhat = np.dot(self.layers[i].weights,x)
    if i!= self.layers.shape[0]:
      yhat = self.layers[i].activation(yhat)
    self.layers[i].yj = yhat
    return yhat

  def _update_weights(self,yhat,tr):
    fiprim = self.layers[-1].d_activation(self.layers[-1-1].yj)
    fiprim = np.expand_dims(fiprim,1).T
    self.layers[-1].delta  = (fiprim * (self.desiered_labels[tr]-self.layers[-1].yj))
    # print(self.desiered_labels[tr]-self.layers[-1-1].yj)
    for i in range(len(self.layers)-2,-1,-1):
      self._delta(i,tr)
    for i in range(len(self.layers)-1,-1,-1):
      yj =np.expand_dims(self.layers[i].yj,1)
      delta = self.layers[i].delta
      deltaw = self.lr * delta * yj
      # print(delta)
      self.layers[i].weights = self.layers[i].weights + deltaw

  def _delta(self,i,tr):
    if i == 0: 
      input = self.training[tr]
    else:
      input = self.layers[i-1].yj
    fiprim =  self.layers[i-1].d_activation(input)
    fiprim = np.expand_dims(fiprim,1)
    sum_delta_w = np.sum((self.layers[i+1].weights*self.layers[i+1].delta).T,1)
    self.layers[i].delta =  (fiprim * sum_delta_w).T

  def Test(self,test_images,testing_label):
    try:
      testings = test_images.reshape(test_images.shape[0],test_images.shape[1]*test_images.shape[2])
    except:
      testings = test_images
    for t in range(0,testings.shape[0]):
      yhat = self._layer_output(testings[t] ,0)
      for i in range(1,self.layers.shape[0]):
        yhat = self._layer_output(yhat ,i)
      print(yhat , testing_label[t])

In [143]:
a = Model(0.01,10)
a.Dense(4,'relu')
a.Dense(4,'relu')
a.Dense(1,'relu')
a.Train(x[10:40],x[10:40])
a.Test(x[10:40], x[10:40])
# to_categorical(training_y, num_classes)

10
9
8
7
6
5
4
3
2
1
[[0.38721508]] 0.387
[[0.40724766]] 0.407
[[0.28167759]] 0.2815
[[0.59853435]] 0.598
[[0.22622155]] 0.2261
[[0.08550491]] 0.08545
[[0.32687304]] 0.3267
[[0.43754082]] 0.4373
[[0.24136814]] 0.2412
[[0.97573314]] 0.975
[[0.3671825]] 0.367
[[0.55798059]] 0.5576
[[0.90488621]] 0.9043
[[0.66351808]] 0.663
[[0.72899041]] 0.7285
[[0.85993506]] 0.8594
[[0.71921842]] 0.7188
[[0.65374609]] 0.6533
[[0.76416957]] 0.7637
[[0.47271998]] 0.4724
[[0.77931615]] 0.779
[[0.79446273]] 0.794
[[0.1106067]] 0.11053
[[0.97035854]] 0.9697
[[0.39723137]] 0.397
[[0.16087137]] 0.1608
[[0.15085509]] 0.1508
[[0.76954416]] 0.769
[[0.20618898]] 0.206
[[0.01005446]] 0.01005


In [ ]:
#show classificaion and error
import matplotlib.pyplot as plt

nerr =np.arange(0,a.err.shape[0])
print(a.err.shape[0])
plt.plot(nerr,a.err,'r')
plt.show()